In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
import mysql.connector
from datetime import datetime
import warnings
import spacy

In [2]:
host = 'localhost'
user = 'root'
password = 'Test@1234'
database = 'github'

# Establish connection
connection = mysql.connector.connect(
    host=host,
    user=user,
    password=password,
    database=database
)

In [3]:
commit_message_english_query = """
                                SELECT r.repo_id, r.langauge, c.messages 
                                FROM repositories r
                                INNER JOIN commit_messages c 
                                    ON r.repo_id = c.repo_id
                                LEFT JOIN commit_message_category ch
                                        ON ch.repo_id = c.repo_id
                                        AND ch.messages = c.messages
                                WHERE
                                    c.messages REGEXP '^[A-Za-z0-9[:space:][:punct:]]*$'
                                    AND ch.messages IS NULL;
"""
df_english_message = pd.read_sql(commit_message_english_query, connection)

print(df_english_message.shape)

/var/folders/jw/rknyb_hs2b50h1tpl5kq9_280000gp/T/ipykernel_33521/2010822267.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_english_message = pd.read_sql(commit_message_english_query, connection)


(91558, 3)


In [4]:
# repeated error categories

rep_error_cat = {
    "Integration Errors": ["merge", "conflict", "integration", "merge conflict"],
    "Compilation Errors": ["compile", "build", "syntax", "error"],
    "Testing Errors": ["test", "failure", "update test", "test case"],
    "Deployment Errors": ["deploy", "deployment", "deployment error", "deploy issue"],
    "Code Quality Issues": ["code quality", "refactor", "code smell", "performance", "security vulnerability"],
    "Environment Inconsistencies": ["environment", "configuration", "mismatch"],
    "Manual Configuration Errors": ["manual configuration", "config error", "server settings"],
    "Rollback Errors": ["rollback", "revert", "undo", "fix rollback"]
}


# overall commit categories
all_error_cat = {
    "Bug Fixes": ["fix", "bug", "issue"],
    "New Features": ["add", "feature"],
    "Performance Improvements": ["refactor", "optimize", "performance"],
    "Security Updates": ["security", "update"],
    "UI/UX Changes": ["ui", "ux", "design"]
}

In [5]:
nlp = spacy.load("en_core_web_sm")

def categorize_message(messages):
    doc = nlp(messages.lower())
    for category, keywords in all_error_cat.items():
        for keyword in keywords:
            if keyword in [token.text for token in doc]:
                return category
            
    return "Others"



def categorize_repeated_message(messages):
    doc = nlp(messages.lower())
    for category, keywords in rep_error_cat.items():
        for keyword in keywords:
            if keyword in [token.text for token in doc]:
                return category
            
    return "Others"

In [6]:
df_english_message['all_category'] = df_english_message['messages'].apply(categorize_message)

df_english_message['repeated_category'] = df_english_message['messages'].apply(categorize_repeated_message)

In [7]:
cursor = connection.cursor()

# # Create table schema
# create_table_query = '''
# CREATE TABLE IF NOT EXISTS commit_message_category (
#     repo_id INT,
#     language VARCHAR(255),
#     messages TEXT,
#     all_category VARCHAR(255),
#     repeated_category VARCHAR(255)
# )
# '''

# # Execute table creation query
# cursor.execute(create_table_query)

In [8]:
for index, row in df_english_message.iterrows():
    insert_query = '''
    INSERT INTO commit_message_category (repo_id, messages, all_category, repeated_category)
    VALUES (%s, %s, %s, %s)
    '''
    cursor.execute(insert_query, (row['repo_id'], row['messages'], row['all_category'], row['repeated_category']))

# Commit changes
connection.commit()

In [9]:
connection.close()